In [1]:
path = '/home/kanit/mimic3/DB_22000_3'
block = 0

In [2]:
from utils import *
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [3]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


In [4]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']
allids = valid_input+valid_output+valid_chart+valid_chart_cate+valid_lab+valid_lab_num+valid_lab_cate+valid_microbio+valid_prescript

In [5]:
print 'len(valid_input) = '+ str(len(valid_input))
print 'len(valid_output) = '+ str(len(valid_output))
print 'len(valid_chart) = '+ str(len(valid_chart))
print 'len(valid_chart_cate) = '+ str(len(valid_chart_cate))
print 'len(valid_lab) = '+ str(len(valid_lab))
print 'len(valid_lab_num) = '+ str(len(valid_lab_num))
print 'len(valid_lab_cate) = '+ str(len(valid_lab_cate))
print 'len(valid_microbio) = '+ str(len(valid_microbio))
print 'len(valid_prescript) = '+ str(len(valid_prescript))
print '\nlen(allids) = '+ str(len(allids) )

len(valid_input) = 277
len(valid_output) = 68
len(valid_chart) = 660
len(valid_chart_cate) = 1
len(valid_lab) = 372
len(valid_lab_num) = 54
len(valid_lab_cate) = 231
len(valid_microbio) = 9085
len(valid_prescript) = 2371

len(allids) = 13119


In [6]:
valid_input_txt =""
for t in valid_input:
    valid_input_txt += str(t)+","
valid_input_txt=valid_input_txt[0:-1]

valid_output_txt =""
for t in valid_output:
    valid_output_txt += str(t)+","
valid_output_txt=valid_output_txt[0:-1]

valid_chart_txt =""
for t in valid_chart:
    valid_chart_txt += str(t)+","
valid_chart_txt=valid_chart_txt[0:-1]

valid_chart_cate_txt =""
for t in valid_chart_cate:
    valid_chart_cate_txt += str(t)+","
valid_chart_cate_txt=valid_chart_cate_txt[0:-1]

valid_lab_txt =""
for t in valid_lab:
    valid_lab_txt += str(t)+","
valid_lab_txt=valid_lab_txt[0:-1]

valid_lab_num_txt =""
for t in valid_lab_num:
    valid_lab_num_txt += str(t)+","
valid_lab_num_txt=valid_lab_num_txt[0:-1]

valid_lab_cate_txt =""
for t in valid_lab_cate:
    valid_lab_cate_txt += str(t)+","
valid_lab_cate_txt=valid_lab_cate_txt[0:-1]

valid_microbio_txt ="'"
for t in valid_microbio:
    valid_microbio_txt += str(t)+"','"
valid_microbio_txt=valid_microbio_txt[0:-2]

valid_prescript_txt ="'"
for t in valid_prescript:
    valid_prescript_txt += str(t)+"','"
valid_prescript_txt=valid_prescript_txt[0:-2]

In [7]:
index =0
map_itemid_index = {}
allitem = valid_output + valid_input + valid_chart + valid_chart_cate + valid_lab + valid_lab_num + valid_lab_cate + valid_microbio + valid_prescript
allitem_unit = ['NOCHECK']*len(valid_output) + valid_input_unit + valid_chart_unit +['NOCHECK'] + valid_lab_unit + valid_lab_num_unit+ ['NOCHECK']*len(valid_lab_cate)+ ['NOCHECK']*len(valid_microbio) + valid_prescript_unit
for i in range(len(allitem_unit)):
    allitem_unit[i] = allitem_unit[i].replace(' ','').lower()
assert len(allitem) == len(allitem_unit)
for ai in allitem:
    map_itemid_index[ai] = index
    index+=1

In [23]:
features = [224828,223835,227443,224697,220235,220224,220339,224700,50819,223830,220274,227037,50820,224695,224685,224686,224684,50826]
file = open(path+'/log/stdout','w')
total = [];
count = [];
for i in range(0,24000):
    try:
        data = np.load(path+'/adm-'+str("%.6d.npy" % i)).tolist()
    except:
        file.write(str(i)+'not found\n');
        continue;
    mat = data['timeseries']
    mat = coodecode(mat['codes'],mat['features'],mat['timestep'])
    ans = []
    for t in mat:
        ts = []
        add = 0;
        for f in features:
            ts.append(t[map_itemid_index[f]])
            if(t[map_itemid_index[f]] != None) : 
                add+=1
        ts.append(t[-2])
        ts.append(t[-1])
        
        if(add>0):
            ans.append(ts)
    
    if(len(ans) > 0):
#         print ans
        data['timeseries'] = ans;
        total.append(data);
        file.write(str(i)+' Done\n')
        count.append(len(data['timeseries']))
    else:
        file.write(str(i)+'discarded zero timestep\n')

    file.flush()
print 'DONE'

DONE


In [25]:
np.save(path+'vent_04042016',total)

In [24]:
total[0]

{'general': [185131,
  29052,
  24715.46736111111,
  15921.0,
  0,
  0,
  3,
  0,
  1,
  4,
  3,
  0,
  4],
 'icd9': [[185131, '51881', '518.81', 7],
  [185131, '5070', '507.0', 7],
  [185131, '5990', '599.0', 9],
  [185131, '5854', '585.4', 9],
  [185131, '41071', '410.71', 6],
  [185131, '5849', '584.9', 9],
  [185131, '42833', '428.33', 6],
  [185131, '28521', '285.21', 3],
  [185131, '2767', '276.7', 2],
  [185131, '25000', '250.00', 2],
  [185131, '40390', '403.90', 6],
  [185131, '4280', '428.0', 6],
  [185131, 'V1251', 'V12.51', 19],
  [185131, 'V5861', 'V58.61', 19],
  [185131, 'V441', 'V44.1', 19],
  [185131, 'V4581', 'V45.81', 19]],
 'timeseries': [[2.0,
   None,
   None,
   None,
   41.0,
   447.0,
   None,
   None,
   None,
   7.42,
   None,
   None,
   7.42,
   None,
   None,
   None,
   None,
   None,
   6900.0,
   185131],
  [0.0,
   None,
   None,
   None,
   32.0,
   418.0,
   None,
   None,
   None,
   7.46,
   None,
   None,
   7.46,
   None,
   None,
   None,
   Non

In [28]:
count = sorted(count)

In [34]:
for t in total:
    if(len(t['timeseries']) > 70):
        print t['timeseries'][0][-1],len(t['timeseries'])


164444 102
149609 79
193878 137
186894 127
186618 113
112235 221
158793 370
192742 154
133430 80
122850 77
113295 72
157942 333
179324 261
182054 199
176653 180
190460 3720
196446 103
160776 262
198412 177
147281 133
115115 247
180458 103
165128 135
182019 91
123110 91
189821 73
179581 84
109999 143
148017 192
147623 93
100416 176
179696 173
197876 98
112325 94
182755 123
185057 76
120508 180
180160 77
186366 96
132401 183
111158 156
185018 221
166467 118
108789 145
103219 227
186020 96
109818 205
198760 104
186666 224
142555 281
132723 108
182954 112
142712 244
117940 206
136829 147
161564 74
138943 122
132582 259
180716 133
128038 74
178842 174
132104 90
191996 158
156655 242
192782 219
103446 126
154710 85
124645 129
150486 130
168578 105
158616 142
183676 330
171493 376
153420 109
137430 72
180858 89
118513 199
118249 142
180714 83
197999 71
122851 161
159469 138
108433 93
137876 164
111103 260
104326 193
175601 107
166548 116
156801 84
129244 724
163023 107
144522 84
179544 870
18

In [ ]:
a=2


In [ ]:
print a+2